In [ ]:
import mne

#data_file='/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003483/sub-009/ses-1/meg/sub-009_ses-1_task-deduction_run-1_meg.fif'
#(shows events in both cases pretty accurate on this one. average for both is similar)

#data_file='/Volumes/M2_DATA/MEG_QC_stuff/data/from lab/ck06an19_vp13/190521/vp13_block1-1.fif'
#(also mostly ok, some false positives. average for both is similar)

#data_file='/Volumes/M2_DATA/MEG_QC_stuff/data/from lab/forrest_gump_meg/en04ns31_vp15/190524/vp15_block1-1.fif'
#(mostly ok not all). average event is a mess, but equal in both cases


data_file='/Volumes/M2_DATA/MEG_QC_stuff/data/from lab/Nilo/sub-01_run-postBase_meg.fif/'
#(bad. average nothing in both  cases)


# data_file = '/Volumes/M2_DATA/MEG_QC_stuff/data/from openneuro/ds003703/sub-a68d5xp5/meg/sub-a68d5xp5_task-listeningToSpeech_run-01_meg.fif'
#(bad. average epoch looks like nothing in both cases. smth is wrong with data?)



raw = mne.io.read_raw_fif(data_file)

picks_ECG = mne.pick_types(raw.info, ecg=True)
ecg_ch=raw.info['chs'][picks_ECG[0]]['ch_name']
print(ecg_ch)



In [ ]:
import plotly.graph_objects as go
import numpy as np

raw_cropped = raw.copy()
tmin=0 
tmax=240
duration = tmax-tmin
raw_cropped.crop(tmin=tmin, tmax=tmax)

ecg_events, ch_ecg, average_pulse, ecg=mne.preprocessing.find_ecg_events(raw_cropped, return_ecg=True, verbose=False)
print('Number  of ECG events detected with ECG channel present: ', len(ecg_events))


#Plot the found ECG events on one of the channels:
mag_ch_names = raw.copy().pick_types(meg='mag').ch_names if 'mag' in raw else None
grad_ch_names = raw.copy().pick_types(meg='grad').ch_names if 'grad' in raw else None
channels = {'mags': mag_ch_names, 'grads': grad_ch_names}

data_mags=raw.get_data(picks = channels['mags'])
data_grads=raw.get_data(picks = channels['grads'])

one_data_channel = data_mags[10]

dt=duration/len(one_data_channel) #take 1 random mag channel, for example 10

t=np.arange(tmin, tmax, 1/raw.info['sfreq'])

fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=one_data_channel, name='data'));
#fig.add_vline(x=ecg_events_times, line_width=1, line_dash="dash", line_color="green")

fig.update_layout(
    title={
    'text': "ECG events on data of 1 magnetometer",
    'y':0.85,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'},
    xaxis_title="Time in sec",
    yaxis = dict(
        showexponent = 'all',
        exponentformat = 'e'))
        
ecg_events_times  = (ecg_events[:, 0] - raw.first_samp) / raw.info['sfreq']

for ecg_t in ecg_events_times:
    fig.add_vline(x=ecg_t, line_width=1, line_color="green")




In [ ]:
raw_cropped2 = raw.copy()
raw_cropped2.crop(tmin=tmin, tmax=tmax)

raw_cropped2.drop_channels(ecg_ch)

ecg_events2, ch_ecg2, average_pulse2, ecg=mne.preprocessing.find_ecg_events(raw_cropped2, return_ecg=True, verbose=False)
print('Number of ECG events detected - ECG reconstructed: ', len(ecg_events2))

ecg_events_times2  = (ecg_events2[:, 0] - raw.first_samp) / raw.info['sfreq']

for ecg_t in ecg_events_times2:
    fig.add_vline(x=ecg_t, line_width=1, line_color="red", line_dash="dash")

fig.show()


In [ ]:
ecg_epochs = mne.preprocessing.create_ecg_epochs(raw_cropped)
fig_ecg = ecg_epochs.plot_image(combine='mean') #plot averageg over ecg epochs artifact

ecg_epochs2 = mne.preprocessing.create_ecg_epochs(raw_cropped2)
fig_ecg2 = ecg_epochs.plot_image(combine='mean') #plot averageg over ecg epochs artifact


In [ ]:

ecg_epochs = mne.preprocessing.create_ecg_epochs(raw_cropped)
ecg_epochs.plot_image(combine='mean')